In [ ]:
(c) Copyright Rosetta Commons Member Institutions.
(c) This file is part of the Rosetta software suite and is made available under license.
(c) The Rosetta software is developed by the contributing members of the Rosetta Commons.
(c) For more information, see http://www.rosettacommons.org. Questions about this can be
(c) addressed to University of Washington CoMotion, email: license@uw.edu.

## Older PyRosetta bindings

## Active Site Energy RMSD Calculation

The goal here is to be able to calculate and energy vs rmsd plot (like what is done for protein folding) to look for a funnel (which indicates convergence of sampling). Here, we want to look at the sampling in the active site by comparing the crystal structure 5EAT to a model made using other templates.

This will take the follwing steps to complete:

1. We must read in the 2 proteins
2. We must be able to access the ligand information in the crystal structure to be able to define the active site
3. We must then overlay the proteins (TMalign)
4. Then we need the sequence mapping from crystal to model (because I didn't do any cleanup)  
5. Once we have the sets of equivalent residues, we can loop over and calculate the rms   
6. Using the same set, we can pull from our energy object the per residue energies for the active site residues
7. Plot!


---


## Step 1 - Get your Rosetta engine's running!


In [ ]:
from rosetta import *

Now that we've imported rosetta, we need to start it. You can pass in *any* command line arguments here to rosetta.

In [ ]:
rosetta.init('-remember_unrecognized_res T -ignore_waters T -extra_res_fa X00.params')

Here's how to construct a pose from a file

In [ ]:
crystal = pose_from_file('5eat.pdb')

In [ ]:
info = crystal.pdb_info()
print info.get_unrecognized_atoms()

In [ ]:
model = pose_from_file('S_0009.pdb')

Let's check out what our pose objects look like. Note: you can verify they are of type Pose by using the pythonic type(crystal)

In [ ]:
print type(crystal)

In [ ]:
print crystal # Note, if you don't want the waters, add the flag -ignore_waters T and restart the rosetta.init

So by default, we get the residues, sequence and fold tree

In [ ]:
print model

So it looks like there are 3 extra residues in the crystal structure (+ the ligand which is stored somewhere). Let's find the ligand data....

---

## Step 2 - Get ligand information from pose

#### Note: EXAMPLE

This section is  a perfect example of interactively finding the information you need from rosetta objects. Start with the pose.pdb_info() object. Store that object to  a new object to get autocompletion to work. Then step into that object and find useful sounding functions (like anything with unrecognized atoms ). The loop over the UA records, store the objects and after executing the cell, we should be able to autocomplete those too!

Task - 

*Start* with pdbinfo = crystal.pdb_info()

*End* with a list of the residues that are in our active site (numbered by the crystal pose)

In [ ]:
crystalpdbinfo = crystal.pdb_info()
ua = crystalpdbinfo.get_unrecognized_atoms()
for i in ua:
    print i.coords()

In [ ]:
### This could be improved with CA selections, (or using Rosetta selections/taskoperations)

crystalpdbinfo = crystal.pdb_info()
print crystalpdbinfo.get_num_unrecognized_atoms()


ua = crystalpdbinfo.get_unrecognized_atoms()

activesiteresnumberset = set()
cutoff = 8

for i in ua:
    ligcoords = i.coords()
    for j in range(1,crystal.n_residue()+1):
        resj = crystal.residue( j )
        for k in resj.atoms():
            disttolig = ligcoords.distance( k.xyz() )  ## Note, type information and methods are in the .hh files in rosetta
            if disttolig < cutoff:
                activesiteresnumberset.add(int(j) )  

print activesiteresnumberset

In [ ]:
print len(activesiteresnumberset)

In [ ]:
mycrystalpdbnumberset = set()
for i in list(activesiteresnumberset):
    mycrystalpdbnumberset.add(crystalpdbinfo.pose2pdb(i) )
    print "Pose Number %s <-----> Crystal Number %s" %(i,crystalpdbinfo.pose2pdb(i))

In [ ]:
## Side bar on comparing with Pymol selection: Sort for readability
#mylist = list(mycrystalpdbnumberset)
#sorted(mylist,key=lambda x: int(x[:3]))

---

## Step 3 - Overlay the proteins for the rms calc

Here, we will just use Tmalign (in Pyrosetta.... see c++ (hybridize.cc for example translation). This previously did not work in pyrosetta because one of the functions called needed to be passed a reference, this has been fixed, for details see <https://github.com/RosettaCommons/main/pull/968> as an example of fixing these kinds of errors. In this case, we simple write another function signature without the need to use the reference. We alternatively could have made getters/setters for the object or we could have written the tmalign(pose1, pose2) into the c++ and called it from the util.hh

For the sake of brevity, these details are left to the Pyrosetta : TMalign tutorial

We are also now going to turn on the Pymol observer and scorefunction so that we can visualize the movements

In [ ]:
from sjb_util import tmalign

pymovercryst = PyMOL_Mover() 
pymovercryst.apply(crystal)
pymovercryst.keep_history(True)
pymovermodel = PyMOL_Mover()

Now you should be able to see the crystal structure in Pymol
Let's add another pymol mover to the model

In [ ]:
pymovermodel = PyMOL_Mover() 
pymovermodel.apply(model)
pymovermodel.keep_history(True)

In [ ]:
from sjb_util import tmalign
atommap , tm = tmalign(model,crystal)

In [ ]:
pymovermodel.apply(model)

---

## Step 4 - Map the active site residues in the crystal to the residues in the model



In [ ]:
print atommap

This next cell is more about seeing what's in the atomid_map< atomid > object. Because We tmaligned model onto pose, the map jth value referes to the CA of the model and we have to atommap.get(jthmodelvalue) to get the crystal value.

In [ ]:
for i in range(1,model.n_residue()+1):
    if model.residue(i).is_protein():
        atom_id_CA_resi = rosetta.core.id.AtomID( model.residue(i).atom_index("CA"),i)
        print "Model CA %s  maps to Crystal CA %s " %(atom_id_CA_resi.rsd(), atommap.get(atom_id_CA_resi).rsd())

In [ ]:
model_map_to_crystal = dict()
for i in list(activesiteresnumberset):
    try:
        if model.residue(i).is_protein():
            atom_id_CA_resi = rosetta.core.id.AtomID( model.residue(i).atom_index("CA"),i)
            print "Model CA %s  maps to Crystal CA %s " %(atom_id_CA_resi.rsd(), atommap.get(atom_id_CA_resi).rsd())
            model_map_to_crystal[atom_id_CA_resi.rsd()] = atommap.get(atom_id_CA_resi).rsd()
    except:
        print "ERROR %s" %i # Dies on the metal ions
        pass

## Step 5 - Calculate RMS

### Manual RMS - ()?

In [ ]:
#print model_map_to_crystal
running_activesite_rms = 0.0
natoms = 0.0

running_activesite_rms_lig = 0.0
nligatoms = 0.0

running_activesite_rms_ca = 0.0
ncaatoms = 0.0

for i in model_map_to_crystal.iterkeys():   ## iter over the active site residues
    modelresi = model.residue(i)
    crystalresi = crystal.residue(model_map_to_crystal[i]) # Get the crystal mapped residue
    assert modelresi.name3() == crystalresi.name3(), "These are different residues~!"
    for j in range(1,crystalresi.natoms()):     # loop over atoms in crystal
        if not crystalresi.atom_is_hydrogen( j ):  #skip the hydrogens
            atomcrystal = crystalresi.atom(j)   

            for l in range(1,modelresi.natoms()): #loop over atoms in model
                if not modelresi.atom_is_hydrogen( l ): #skip hydrogens
                    atommodel = modelresi.atom(l)
                    
                    
                    if (crystalresi.atom_name(j) == modelresi.atom_name(l)): # same type of atom
                        rms_j_l = atommodel.xyz().distance( atomcrystal.xyz() )
                   
                        running_activesite_rms += rms_j_l
                        natoms += 1.0
                        
                        # Ca active site rsmd counters here
                        if modelresi.atom_name(j) == ' CA ':
                        
                            rms_jca_lca = atommodel.xyz().distance( atomcrystal.xyz() )
                            running_activesite_rms_ca += rms_jca_lca
                            ncaatoms +=1
                        
print "Over %s atoms in active site" %natoms
print (running_activesite_rms)/(natoms)

print "Over %s CA atoms  in active site " %ncaatoms #this should be 55 - 3 metals
print (running_activesite_rms_ca)/(ncaatoms)

## There's our RMS

To review, since this is a bit messy, we are hand calculating the rms for the heavy atoms that we define as our active site. We loop over the residue numbers and access the Residue() objects. We make sure the residue names are the same. Then we access each Residue().Atom(i) object by value. We remove hydrogens by checking the Residue level object using the method atom_is_hydrogen(i) which is indexed the sames. The reason we do this is because the Atom object that is attached here lacks a lot of information.

A better way to do this would be to lookup the atom type based on the Residue().Atom(i).type() and see if it's a hydrogen.

---
## Step 6 - Energy for the Active site residues

Now we have the rms, we just need to access the energies for those residues and add them up!
We will do this using a score method (see the Scoring Pyrosetta Tutorial) for details. But basically this populates a pandas Dataframe with the indices equal to the residue number (of the model, which is what we care about)


In [ ]:
import pandas as pd
from rosetta.core.scoring.methods import EnergyMethodOptions

sfxn = get_fa_scorefxn()  #get a scorefunction (default talaris2013)
talaris2013_energy_methods = sfxn.energy_method_options() #have to copy the default energy methods from talaris first
emo = EnergyMethodOptions( talaris2013_energy_methods)    #must do this to get per res hbond_bb terms in breakdown
emo.hbond_options().decompose_bb_hb_into_pair_energies( True )  # set to true, defaults False
sfxn.set_energy_method_options( emo ) #set the sfxn up with the energy method options
print sfxn(model)

score_types = []
for i in range(1, rosetta.core.scoring.end_of_score_type_enumeration+1):
    ii = rosetta.core.scoring.ScoreType(i)
    if model.energies().weights()[ii] != 0: score_types.append(ii)
        
listofseries = []
for j in range(1,model.total_residue()+1):
    mydict = {}
    for i in score_types:
        myweight = model.energies().weights()[i]
        mydict['%s' %core.scoring.ScoreTypeManager.name_from_score_type(i)] = myweight*model.energies().residue_total_energies(j)[i]

    listofseries.append( pd.Series(mydict))

df = pd.DataFrame(listofseries)
df.index +=1 #makes index start at 1, not 0. Now, each row refers to its proper residue number (ie resi =1 -> row1)
df = df.T # Need to do this to be able to access per resi energy with df[[1]] indexing
print model.n_residue() # Just to check the lengths are the same

Now we just subselect the active site residues from the df object

In [ ]:
# Then just add a .sum(), okay, now just add 1 more .sum()
df[ [x for x in model_map_to_crystal.iterkeys()] ].sum().sum()

## There's our active site energy!

Now we could wrap this all into a nice function, add some tests, and run on all 10000 docking trajectories.